In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import nltk
import pandas as pd
import os
from eunjeon import Mecab
from ast import literal_eval

xname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#8_clean_text'}.xlsx")
df = pd.read_excel(xname, index_col=0)
mecab = Mecab()
corpus = []
TEXT = []
count = 0

for i in range(len(df)):
    text = literal_eval(df.loc[i, 'text'])
    corpus.append( ". ".join([" ".join([f for f in mecab.nouns(e) if len(f)>1]) for e in text]))
     
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(corpus)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

for i, sent in enumerate(corpus):
    
    seen = set()
    result =  [ token for token in sent.split() if sp_matrix[i, word2id[token]] > 0.02 ]
    result =  [x for x in result if not (x in seen or seen.add(x))]
    TEXT.append(result)

df['text'] = TEXT
x2name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#9_tf_idf'}.xlsx")
df.to_excel(x2name)